<a href="https://colab.research.google.com/github/johnnyff/naver_boostcourse_pytorch/blob/master/pytorch_boostcourse_lab10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.duca.manual_seed_all(777)

In [ ]:
#parameters
learning_rate = 0.001
training_epochs =15
batch_size = 100


In [ ]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                         transform = transforms.ToTensor(),
                         download =True)
mnist_test = dsets.MNIST(root = "MNIST_data/",
                        train= False,
                        transform =transforms.ToTensor(),
                        download =True)

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                            drop_last=True,
                                          num_workers = 2
                                          )


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1= nn.Sequential(
        nn.Conv2d(1,32,kernel_size= 3 , stride =1, padding =1),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
        nn.Conv2d(32,64, kernel_size= 3, stride =1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size =3 , stride=1, padding =1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1= nn.Linear(3*3*128,625 )
        self.relu = nn.ReLU()
        self.fc2= nn.Linear(625, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0),-1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)

        return out

In [ ]:
model = CNN().to(device)


In [ ]:
value = torch.Tensor(1,1,28,28).to(device)
print((model(value)).shape)

torch.Size([1, 10])


In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device) #이미지
        Y = Y.to(device ) # label
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost /total_batch
    print('[Epoch : {}] cost = {}'.format(epoch+1, avg_cost))

[Epoch : 1] cost = 0.16316992044448853
[Epoch : 2] cost = 0.04148160666227341
[Epoch : 3] cost = 0.02944965846836567
[Epoch : 4] cost = 0.021708160638809204
[Epoch : 5] cost = 0.016919493675231934
[Epoch : 6] cost = 0.015364393591880798
[Epoch : 7] cost = 0.011557841673493385
[Epoch : 8] cost = 0.00978616438806057
[Epoch : 9] cost = 0.009146269410848618
[Epoch : 10] cost = 0.009341293014585972
[Epoch : 11] cost = 0.006328644696623087
[Epoch : 12] cost = 0.006175466813147068
[Epoch : 13] cost = 0.008107916451990604
[Epoch : 14] cost = 0.0059585776180028915
[Epoch : 15] cost = 0.002145378617569804


In [1]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ',accuracy.item() )

NameError: ignored